# Capacity Building

## Previous notebooks
The previous notebooks (01 to 03) have demonstrated the general approach to building a compartmental model using *summer*, adding flows, requesting derived outputs and applying straticfications.

## This notebook
This notebook builds off notebook 03, which provided a broader introduction to stratification.
It applies a more limited range of stratifications to a compartmental model and demonstrates the process of manually calibrating this model to empiric data.

## Data inputs
### Imports
Install the summer package, as per previous notebooks

In [ ]:
# If we are running in google colab, pip install the required packages, 
# but do not modify local environments
try:
  import google.colab
  IN_COLAB = True
  %pip install summerepi
except:
  IN_COLAB = False

In [ ]:
# Python standard library imports come first
from datetime import datetime, timedelta
from typing import List

# Then external package imports
import pandas as pd

# Explicit imports from the summer modelling package
from summer import CompartmentalModel
from summer import Stratification

# Set pandas to use our favourite interactive plotting tool
pd.options.plotting.backend = "plotly"

# Define constants and variables

# Define an arbitrary reference date,
# because we will need numbers (not dates) to go into the model solver
COVID_BASE_DATE = datetime(2019, 12, 31)

# Will need this for indexing the MoH data later
region = "Malaysia"

### Utility functions
Data downloads have been moved to a separate module, which we will now obtain

In [ ]:
# If running in google colab, download the required python module
if IN_COLAB:
    !wget https://raw.githubusercontent.com/monash-emu/AuTuMN/master/notebooks/capacity_building/malaysia/get_mys_data.py

import get_mys_data

### Get data
Now call the functions from our imported module to obtain the required data

In [ ]:
df = get_mys_data.fetch_mys_data()
initial_population = get_mys_data.get_initial_population(region)
observations = get_mys_data.get_target_observations(df, region, "cases")

In [ ]:
start_date = datetime(2021, 1, 1)  # Define the model's start date
end_date = start_date + timedelta(days=300)  # Define the model's duration

# Numeric representation of the start and end dates
start_date_int = (start_date - COVID_BASE_DATE).days
end_date_int = (end_date - COVID_BASE_DATE).days

In [ ]:
# Define a target set of observations to compare against our modelled outputs later
notifications_target = observations[start_date: end_date]["cases_new"]

## Model
### Define a model

In [ ]:
def build_unstratified_model(base_compartments: List[str], infectious_seed: float, parameters: dict) -> CompartmentalModel:
    """
    Create a compartmental model, with the minimal compartmental structure needed to run and produce some sort of 
    meaningful outputs.
    
    Args:
        base_compartments: The names of the base (unstratified compartments)
        infectious_seed: The number of infectious persons to start the epidemic
        parameters: Flow parameters
    
    """

    model = CompartmentalModel(
        times=(start_date_int, end_date_int),
        compartments=base_compartments,
        infectious_compartments=["I"],
        ref_date=COVID_BASE_DATE
    )

    model.set_initial_population(distribution={"S": initial_population - infectious_seed, "E": 0, "I": infectious_seed})
    
    # Susceptible people can get infected
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=parameters["contact_rate"], 
        source="S", 
        dest="E"
    )
    # Expose people transition to infected
    model.add_transition_flow(
        name="progression",
        fractional_rate=parameters["progression_rate"],
        source="E",
        dest="I",
    )

    # Infectious people recover
    model.add_transition_flow(
        name="recovery",
        fractional_rate=parameters["recovery_rate"],
        source="I",
        dest="R",
    )

    # Add an infection-specific death flow to the I compartment
    model.add_death_flow(name="infection_death", death_rate=parameters["death_rate"], source="I")

    # We will also request an output for the 'progression' flow
    model.request_output_for_flow("progressions", "progression")

    return model

## Stratifications
### Define stratifications

In [ ]:
def get_age_stratification(compartments_to_stratify: List[str]) -> Stratification:
    """
    Create a fairly simple summer stratification object that splits all compartments
    according to age into young and old.
    
    Args:
        compartments_to_stratify: List of the compartments to stratify, which should be all the compartments
    Returns:
        A summer stratification object to represent age stratification (not yet applied)
    """
    
    # Create the stratification
    strata=["young", "old"]
    strat = Stratification(name="age", strata=strata, compartments=compartments_to_stratify)

    # Split the population unevenly, with more being old than young
    pop_split = {"young": 0.25, "old": 0.75}
    strat.set_population_split(pop_split)

    return strat

In [ ]:
def get_severity_strat() -> Stratification:
    """
    Create a stratification named 'severity', applying to the infectious, which
    splits that compartment into 'asymptomatic', 'mild' and 'severe'.
    
    Returns:
        A summer stratification object to represent severity stratification (not yet applied)
    """
    
    # Create the stratification object
    severity_strata = ["asymptomatic", "mild", "severe"]
    severity_strat = Stratification(name="severity", strata=severity_strata, compartments=["I"])
   
    return severity_strat

## Adjust the rates of progression to various severities following infection
Because this stratification is applied to the I compartment only, 
and because progression is the flow that enters the I compartment,
these adjustments can also be thought of as "splits"
(i.e. the proportion of new cases progressing to each state).
These parameters are in the loose range of what might be realistic,
but should ideally be informed by a specific study we could reference.

In [ ]:
def add_progression_adjustments(severity_strat: Stratification):
    # Children are mostly asymptomatic and never severe
    young_progression_adjustments = {
        "asymptomatic": 0.7,
        "mild": 0.3,
        "severe": 0.,
    }

    # Adults are a mix, with a minority becoming severe
    old_progression_adjustments = {
        "asymptomatic": 0.5,
        "mild": 0.4,
        "severe": 0.1,
    }
    severity_strat.set_flow_adjustments(
        "progression",
        young_progression_adjustments,
        source_strata={"age": "young"},
    )
    severity_strat.set_flow_adjustments(
        "progression",
        old_progression_adjustments,
        source_strata={"age": "old"},
    )

## Adjust the infectiousness of the various states
Note that this means that children and adults will contribute to transmission differently,
but only because they end up in different severity categories.
Assigning `None` to a category means that the unadjusted value (1) will be retained.
(This is necessary because all strata must be specified.)

In [ ]:
def add_infectiousness_adjustments(severity_strat: Stratification):
    all_age_infectiousness_adjustments = {
        "asymptomatic": 0.5,
        "mild": None,
        "severe": 1.5,
    }
    severity_strat.add_infectiousness_adjustments(
        "I",
        all_age_infectiousness_adjustments
    )

In [ ]:
def build_stratified_model(base_compartments: List[str], infectious_seed: float, parameters: dict) -> CompartmentalModel:
    model = build_unstratified_model(base_compartments, infectious_seed, parameters)

    age_strat = get_age_stratification(base_compartments)
    severity_strat = get_severity_strat()

    add_progression_adjustments(severity_strat)
    add_infectiousness_adjustments(severity_strat)

    # Apply the stratifications we developed previously
    model.stratify_with(age_strat)
    model.stratify_with(severity_strat)

    return model

## Outputs

In [ ]:
# Build and run the stratified model
parameters = {
    "contact_rate": 10.,
    "progression_rate": 0.2,
    "recovery_rate": 0.2,
    "death_rate": 0.,
    "reporting_fraction": 0.01,
}
base_compartments = ["S", "E", "I", "R"]
infectious_seed = 100.

model = build_stratified_model(base_compartments, infectious_seed, parameters)
model.run()

In [ ]:
progressions = model.get_derived_outputs_df()["progressions"]
notifications_modelled = progressions * parameters["reporting_fraction"]

In [ ]:
pd.DataFrame(
    {"observed": notifications_target,
    "modelled": notifications_modelled}
).plot()

# Manually calibrating to fit to notifications of Delta Wave

In [ ]:
start_date = datetime(2021, 7, 15)  # Define the model's start date to capture Delta wave
end_date = start_date + timedelta(days=180)  # Define the model's duration

# Numeric representation of the start and end dates
start_date_int = (start_date - COVID_BASE_DATE).days
end_date_int = (end_date - COVID_BASE_DATE).days

In [ ]:
# Define a target set of observations to compare against our modelled outputs later
notifications_target = observations[start_date: end_date]["cases_new"]

In [ ]:
custom_params = {
        "contact_rate": 1.75,
        "progression_rate": 0.2,
        "recovery_rate": 0.2,
        "death_rate": 0.01,
        "reporting_fraction": 0.01,
    }

infectious_seed =100000.

In [ ]:
model = build_stratified_model(base_compartments, infectious_seed, custom_params)
model.run()

In [ ]:
progressions = model.get_derived_outputs_df()["progressions"]
notifications_modelled = progressions * parameters["reporting_fraction"]

In [ ]:
pd.DataFrame(
    {"observed": notifications_target,
    "modelled": notifications_modelled}
).plot()